In [1]:
import matplotlib.pyplot as plt

import numpy as np #importa a biblioteca usada para trabalhar com vetores e matrizes
import pandas as pd #importa a biblioteca usada para trabalhar com dataframes
import util
from scipy.sparse import bsr_matrix

#importa o arquivo e extrai as features
Xfeatures, Y = util.extract_features('datasets/everything.csv', rep='ngrams',n=3)

['lol' 'ol' 'atl' 'tle' 'lea']


In [2]:
import svmutil
from svmutil import svm_read_problem
from svmutil import svm_problem
from svmutil import svm_parameter
from svmutil import svm_train
from svmutil import svm_predict
from svmutil import svm_save_model


In [3]:
### Classificador - Kernel linear ###

# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
Xk, Yk = Xfeatures[idx_perm, :], Y[idx_perm]

# define a porcentagem de dados que irao compor o conjunto de treinamento
pTrain = 0.8 

# obtem os indices dos dados da particao de treinamento e da particao de teste
train_index, test_index = util.stratified_holdOut(Yk, pTrain)

X_train, X_test = Xk[train_index, :], Xk[test_index, :];
Y_train, Y_test = Yk[train_index], Yk[test_index];

train_index, val_index = util.stratified_holdOut(Y_train, pTrain)

Xtrain, Xvalid = X_train[train_index, :], X_train[val_index, :]
Ytrain, Yvalid = Y_train[train_index], Y_train[val_index]



print('Numero de dados de validação: %d' %(Xvalid.shape[0]))

# Treinamento:
custo = 26.0
kernel = 0 # kernel linear
model = svm_train(Ytrain, Xtrain, '-c %f -t %d' %(custo, kernel))

# Predição:
p_labs, p_acc, p_vals = svm_predict(Yvalid, bsr_matrix(Xvalid), model)

Numero de dados de validação: 444
Accuracy = 81.7568% (363/444) (classification)


In [4]:
def gridSearch(X, Y, Xval, Yval, kernel):
    """
    Retorna o melhor valor para os parâmetros custo e gamma do SVM radial.
    
    Parâmetros
    ----------
    X : matriz com os dados de treinamento
    
    y : vetor com classes de cada dados de treinamento
    
    Xval : matriz com os dados de validação
    
    yval : vetor com as classes dos dados de validação
    
    Retorno
    -------
    custo, gamma : os melhores valores para os parêmetros custo e gamma.
    
     """
    
    #inicializa as variáveis que deverão ser retornadas pela função
    custo = 1000
    gamma = 1000
    
    ########################## COMPLETE O CÓDIGO AQUI  ########################
    # Complete esta função para retornar os melhores valores de C e gamma com base
    # nos dados do conjunto de validação.
    # Você pode usar a função svm_predict() para fazer a predição das classes das amostras do
    # conjunto de validação. 
    #
    # Importante
    # ----
    # Você pode calcular a acurácia usando
    # 
    #    acuracia = np.sum(classes==Yval)/len(Yval) 
    
    aux = [0.01,0.1,1,10,100]
    values = []
    acc = 0
    
    for i in range(1,9):
        for number in aux:
            values.append(i * number)
    values.sort()
    
    # Kernel linear
    if(kernel == 0):
        for aux_custo in values:
            model = svm_train(Ytrain, Xtrain, '-c %f -t %d -q' %(aux_custo, kernel))
            p_labs, p_acc, p_vals = svm_predict(Yval, Xval, model)

            if(p_acc[0] > acc):
                acc = p_acc[0]
                custo = aux_custo
    
    # Lista de valores testados para 'custo' e 'gamma' foram escolhidos com base no artigo:
    # https://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf
    
    if(kernel == 1 or kernel == 2):
        for i in range (-5, 10, 2):
            for j in range (-10, 4, 2):
                aux_custo = 2**j
                aux_gamma = 2**i

                model = svm_train(Y, X, '-c %f -t %d -g %f -q' %(aux_custo, kernel, aux_gamma))
                p_labs, p_acc, p_vals = svm_predict(Yval, Xval, model)

                if(p_acc[0] > acc):
                    acc = p_acc[0]
                    custo = aux_custo
                    gamma  = aux_gamma
    
    ##########################################################################

    return custo, gamma

In [5]:
def imprimeRelatorio(p_acc, kernel):
    print('\n\tAccuracy \tErro quadratico   \tCoeficiente correlacao   \tKernel')
    print('\t%1.3f      \t%1.3f      \t\t%1.3f      \t\t\t%d' % (p_acc[0], p_acc[1], p_acc[2], kernel ) )
    
    print('\t------------------------------------------------')

In [ ]:
bestC_linear, bestGamma_linear = gridSearch(bsr_matrix(Xtrain), Ytrain, bsr_matrix(Xvalid), Yvalid, 0)
print('Melhores parâmetros\nLinear - C: %1.3f\n' %(bestC_linear))

In [ ]:
bestC_polinomial, bestGamma_polinomial = gridSearch(bsr_matrix(Xtrain), Ytrain, bsr_matrix(Xvalid), Yvalid, 1)
print('Melhores parâmetros\nPolinomial - C: %1.3f, gamma=%1.3f' %(bestC_polinomial,bestGamma_polinomial))

In [ ]:
bestC_radial, bestGamma_radial = gridSearch(bsr_matrix(Xtrain), Ytrain, bsr_matrix(Xvalid), Yvalid, 2)
print('Melhores parâmetros\nRadial - C: %1.3f, gamma=%1.3f' %(bestC_radial,bestGamma_radial))

In [ ]:
### Classificador - Kernel radial ### 
# treina o classificador com o melhor custo e o melhor gamma encontrados 
# Treinamento e clasificacao com valores de kernel:

# 0 -- linear: u\'\*v
# 1 -- polynomial: (gamma\*u\'\*v + coef0)^degree
# 2 -- radial basis function: exp(-gamma\*|u-v|^2)

print('\n-----------\nSVM Classifier: \n-----------\n')

# Kernel linear:
model = svm_train(Ytrain, bsr_matrix(Xtrain), '-c %f -t %d' %(bestC_linear, 0))
p_labs, p_acc, p_vals = svm_predict(Yvalid, bsr_matrix(Xvalid), model)
imprimeRelatorio(p_acc, 0)


# Kernel polinomial:
model = svm_train(Ytrain, bsr_matrix(Xtrain), '-q -c %f -t %d -g %f' %(bestC_polinomial, 1, bestGamma_polinomial))
p_labs, p_acc, p_vals = svm_predict(Yvalid, bsr_matrix(Xvalid), model)
imprimeRelatorio(p_acc, 1)

# Kernel raidal:
model = svm_train(Ytrain, bsr_matrix(Xtrain), '-q -c %f -t %d -g %f' %(bestC_radial, 2, bestGamma_radial))
p_labs, p_acc, p_vals = svm_predict(Yvalid, bsr_matrix(Xvalid), model)
imprimeRelatorio(p_acc, 2)
    

In [ ]:
def curva_aprendizado_linear(Xtrain, Ytrain, Xvalid, Yvalid, Cost):
    """
    Funcao usada gerar a curva de aprendizado.
  
    Parametros
    ----------
  
    X : matriz com os dados de treinamento
  
    Y : vetor com as classes dos dados de treinamento
  
    Xval : matriz com os dados de validação
  
    Yval : vetor com as classes dos dados de validação
  
    """

    # inicializa as listas que guardarao a performance no treinamento e na validacao
    perf_train = []
    perf_val = []

    # inicializa o parametro de regularizacao da regressao logistica
    lambda_reg = 1
        
    # Configura o numero de interacaoes da regressao logistica
    iteracoes = 500
        
    # Kernel radial
    kernel = 0
    
    ########################## COMPLETE O CÓDIGO AQUI  ###############################
    #  Instrucoes: Complete o codigo para gerar o gráfico da curva de aprendizado.
    #           Comece o treinamento com as primeiras 10 amostras da base de dados de 
    #           treinamento e calcule a acuracia do classificador tanto nos dados de
    #           treinamento já apresentados, quando na base de validacao. 
    #           Depois disso, adicione mais um dado para treinamento e calcule novamente 
    #           o desempenho. Continue adicionando um dado por vez ate todos os dados de 
    #           treinamento serem usados. Nas listas perf_train e perf_val, guarde a acuracia 
    #           obtida nos dados de treinamento e na base de validacao a cada nova adicao de 
    #           dados para treinamento.
    for idx, i in enumerate(np.arange(9, len(Xtrain) - 1)):
        model = svm_train(Ytrain[:i], bsr_matrix(Xtrain[:i, :]), '-c %f -t %d' %(Cost, kernel))
        
        # Teste
        p_labs, p_acc, p_vals = svm_predict(Ytrain[:i], bsr_matrix(Xtrain[:i, :]), model)
        perf_train.append(p_acc[0])
        
        # Validação
        p_labs, p_acc, p_vals = svm_predict(Yvalid, bsr_matrix(Xvalid), model)
        perf_val.append(p_acc[0])
    
    ##################################################################################
       
    # Define o tamanho da figura 
    plt.figure(figsize=(20,12))

    # Plota os dados
    plt.plot(perf_train, color='blue', linestyle='-', linewidth=1.5, label='Treino') 
    plt.plot(perf_val, color='red', linestyle='-', linewidth=1.5, label='Validação')

    # Define os nomes do eixo x e do eixo y
    plt.xlabel(r'# Qtd. de dados de treinamento',fontsize='x-large') 
    plt.ylabel(r'Acuracia',fontsize='x-large') 

    # Define o título do gráfico
    plt.title(r'Curva de aprendizado', fontsize='x-large')

    # Acrescenta um grid no gráfico
    plt.grid(axis='both')

    # Plota a legenda
    plt.legend()
    
    plt.show()
    
curva_aprendizado_linear(Xtrain, Ytrain, Xvalid, Yvalid, bestC_linear)

In [ ]:
def curva_aprendizado(Xtrain, Ytrain, Xvalid, Yvalid, Cost, Gamma, kernel):
    """
    Funcao usada gerar a curva de aprendizado.
  
    Parametros
    ----------
  
    X : matriz com os dados de treinamento
  
    Y : vetor com as classes dos dados de treinamento
  
    Xval : matriz com os dados de validação
  
    Yval : vetor com as classes dos dados de validação
  
    """

    # inicializa as listas que guardarao a performance no treinamento e na validacao
    perf_train = []
    perf_val = []

    # inicializa o parametro de regularizacao da regressao logistica
    lambda_reg = 1
        
    # Configura o numero de interacaoes da regressao logistica
    iteracoes = 500
    
    ########################## COMPLETE O CÓDIGO AQUI  ###############################
    #  Instrucoes: Complete o codigo para gerar o gráfico da curva de aprendizado.
    #           Comece o treinamento com as primeiras 10 amostras da base de dados de 
    #           treinamento e calcule a acuracia do classificador tanto nos dados de
    #           treinamento já apresentados, quando na base de validacao. 
    #           Depois disso, adicione mais um dado para treinamento e calcule novamente 
    #           o desempenho. Continue adicionando um dado por vez ate todos os dados de 
    #           treinamento serem usados. Nas listas perf_train e perf_val, guarde a acuracia 
    #           obtida nos dados de treinamento e na base de validacao a cada nova adicao de 
    #           dados para treinamento.
    for idx, i in enumerate(np.arange(9, len(Xtrain) - 1)):
        model = svm_train(Ytrain[:i], bsr_matrix(Xtrain[:i, :]), '-q -c %f -t %d -g %f' %(Cost, kernel, Gamma))
        
        # Teste
        p_labs, p_acc, p_vals = svm_predict(Ytrain[:i], bsr_matrix(Xtrain[:i, :]), model)
        perf_train.append(p_acc[0])
        
        # Validação
        p_labs, p_acc, p_vals = svm_predict(Yvalid, bsr_matrix(Xvalid), model)
        perf_val.append(p_acc[0])
    
    ##################################################################################
       
    # Define o tamanho da figura 
    plt.figure(figsize=(20,12))

    # Plota os dados
    plt.plot(perf_train, color='blue', linestyle='-', linewidth=1.5, label='Treino') 
    plt.plot(perf_val, color='red', linestyle='-', linewidth=1.5, label='Validação')

    # Define os nomes do eixo x e do eixo y
    plt.xlabel(r'# Qtd. de dados de treinamento',fontsize='x-large') 
    plt.ylabel(r'Acuracia',fontsize='x-large') 

    # Define o título do gráfico
    plt.title(r'Curva de aprendizado', fontsize='x-large')

    # Acrescenta um grid no gráfico
    plt.grid(axis='both')

    # Plota a legenda
    plt.legend()
    
    plt.show()
    
curva_aprendizado(Xtrain, Ytrain, Xvalid, Yvalid, bestC_polinomial, bestGamma_polinomial, 1)

In [ ]:
curva_aprendizado(Xtrain, Ytrain, Xvalid, Yvalid, bestC_radial, bestGamma_radial, 2)

In [ ]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
Xk, Yk = Xfeatures[idx_perm, :], Y[idx_perm]

nFolds = 10
classes = [0,1]
iteracoes=1000
folds = util.stratified_kfolds(Yk, nFolds, classes) 

k=1
resultados=[]
for train_index, test_index in folds:
    print('\n-----------\n%d-fold: \n-----------\n' % (k) )

    # se train_index ou test_index forem vazios, interrompe o laco de repeticao
    if len(train_index)==0 or len(test_index)==0: 
        print('\tErro: o vetor com os indices de treinamento ou o vetor com os indices de teste esta vazio')      
        break
        
    totalFold = len(train_index)+len(test_index)

    X_train, X_test = Xk[train_index, :], Xk[test_index, :];
    Y_train, Y_test = Yk[train_index], Yk[test_index];
    
    # separa os dados de treinamento em treinamento e validacao
    pTrain = 0.8
    train_index_v, val_index = util.stratified_holdOut(Y_train, pTrain)
    
    #bestC_linear, bestGamma_linear = gridSearch(Xtrain, Ytrain, Xvalid, Yvalid, 0)

    model = svm_train(Ytrain, bsr_matrix(Xtrain), '-c %f -t %d' %(bestC_linear, 0))
    
    Y_pred, p_acc, p_vals = svm_predict(Y_test, bsr_matrix(X_test), model)

    # Compute confusion matrix
    cm = util.get_confusionMatrix(Y_test, np.array(Y_pred).astype(int), classes)

    # Gera o relatório de desempenho
    #print('\n\n\n\t'+"="*50+'\n\tMelhor parametro de regularizacao: %1.6f' %bestRegularization)
    print('\n\tResultado no fold atual usando o melhor parametro encontrado:')
    auxResults = util.relatorioDesempenho(cm, classes, imprimeRelatorio=True)

    # adiciona os resultados do fold atual na lista de resultados
    resultados.append( auxResults ) 
        
    k+=1

In [ ]:
print('\nResultado final da classificação:')
util.mediaFolds( resultados, classes )

In [ ]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
Xk, Yk = Xfeatures[idx_perm, :], Y[idx_perm]

nFolds = 10
classes = [0,1]
iteracoes=1000
folds = util.stratified_kfolds(Yk, nFolds, classes) 

k=1
resultados=[]
for train_index, test_index in folds:
    print('\n-----------\n%d-fold: \n-----------\n' % (k) )

    # se train_index ou test_index forem vazios, interrompe o laco de repeticao
    if len(train_index)==0 or len(test_index)==0: 
        print('\tErro: o vetor com os indices de treinamento ou o vetor com os indices de teste esta vazio')      
        break
        
    totalFold = len(train_index)+len(test_index)

    X_train, X_test = Xk[train_index, :], Xk[test_index, :];
    Y_train, Y_test = Yk[train_index], Yk[test_index];
    
    # separa os dados de treinamento em treinamento e validacao
    pTrain = 0.8
    train_index_v, val_index = util.stratified_holdOut(Y_train, pTrain)
    
    #bestC_polinomial, bestGamma_polinomial = gridSearch(Xtrain, Ytrain, Xvalid, Yvalid, 1)

    model = svm_train(Ytrain, bsr_matrix(Xtrain), '-q -c %f -t %d -g %f' %(bestC_polinomial, 1, bestGamma_polinomial))
    
    Y_pred, p_acc, p_vals = svm_predict(Y_test, bsr_matrix(X_test), model)

    # Compute confusion matrix
    cm = util.get_confusionMatrix(Y_test, np.array(Y_pred).astype(int), classes)

    # Gera o relatório de desempenho
    #print('\n\n\n\t'+"="*50+'\n\tMelhor parametro de regularizacao: %1.6f' %bestRegularization)
    print('\n\tResultado no fold atual usando o melhor parametro encontrado:')
    auxResults = util.relatorioDesempenho(cm, classes, imprimeRelatorio=True)

    # adiciona os resultados do fold atual na lista de resultados
    resultados.append( auxResults ) 
        
    k+=1

In [ ]:
print('\nResultado final da classificação:')
util.mediaFolds( resultados, classes )

In [ ]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
Xk, Yk = Xfeatures[idx_perm, :], Y[idx_perm]

nFolds = 10
classes = [0,1]
iteracoes=1000
folds = util.stratified_kfolds(Yk, nFolds, classes) 

k=1
resultados=[]
for train_index, test_index in folds:
    print('\n-----------\n%d-fold: \n-----------\n' % (k) )

    # se train_index ou test_index forem vazios, interrompe o laco de repeticao
    if len(train_index)==0 or len(test_index)==0: 
        print('\tErro: o vetor com os indices de treinamento ou o vetor com os indices de teste esta vazio')      
        break
        
    totalFold = len(train_index)+len(test_index)

    X_train, X_test = Xk[train_index, :], Xk[test_index, :];
    Y_train, Y_test = Yk[train_index], Yk[test_index];
    
    # separa os dados de treinamento em treinamento e validacao
    pTrain = 0.8
    train_index_v, val_index = util.stratified_holdOut(Y_train, pTrain)
    
    #bestC_polinomial, bestGamma_polinomial = gridSearch(Xtrain, Ytrain, Xvalid, Yvalid, 1)

    model = svm_train(Ytrain, bsr_matrix(Xtrain), '-q -c %f -t %d -g %f' %(bestC_radial, 2, bestGamma_radial))
    
    Y_pred, p_acc, p_vals = svm_predict(Y_test, bsr_matrix(X_test), model)

    # Compute confusion matrix
    cm = util.get_confusionMatrix(Y_test, np.array(Y_pred).astype(int), classes)

    # Gera o relatório de desempenho
    #print('\n\n\n\t'+"="*50+'\n\tMelhor parametro de regularizacao: %1.6f' %bestRegularization)
    print('\n\tResultado no fold atual usando o melhor parametro encontrado:')
    auxResults = util.relatorioDesempenho(cm, classes, imprimeRelatorio=True)

    # adiciona os resultados do fold atual na lista de resultados
    resultados.append( auxResults ) 
        
    k+=1

In [ ]:
print('\nResultado final da classificação:')
util.mediaFolds( resultados, classes )